# GPS tracks (GPX files) to Geopandas / Shapefile
Jupyter Notebook by Florian Neukirchen. Related blog post: https://www.riannek.de/2022/gpx-to-geopandas/

It is easy to open a complete folder of GPS Tracks (GPX files) in Geopandas, e.g. for plotting maps or to export them as shapefile.

In [1]:
import geopandas as gpd
import os

In [2]:
# change statement below to have the path to your apple_health_export/workout-routes directory
folder = "apple_health_export/workout-routes"

In [ ]:
path = os.getcwd()
print("Path of the current directory : " + path)
# get the name of the current directory
repn = os.path.basename(path)
print("Name of the current directory : " + repn)

In [11]:
# Create empty GeoDataFrame
trackpoints = gpd.GeoDataFrame(columns=['time', 'geometry'], geometry='geometry')

Try to open all gpx files in folder. Geopandas uses fiona to open/parse gpx. 

In [13]:
for file in os.listdir(folder):
    if file.endswith(('.gpx')):
        try:
            gdf = gpd.read_file(os.path.join(folder, file), layer='track_points')
            trackpoints = trackpoints.append(gdf[['time', 'geometry']])
        except:
            print("Error", file)

In [14]:
trackpoints.sort_values(by="time", inplace=True)
trackpoints.reset_index(inplace=True, drop=True)

In [15]:
trackpoints.head()

,time,geometry
0,2020-03-13T21:19:33,POINT (-97.74596 30.35053)
1,2020-03-13T21:19:44,POINT (-97.74616 30.35065)
2,2020-03-13T21:19:45,POINT (-97.74617 30.35066)
3,2020-03-13T21:19:46,POINT (-97.74619 30.35067)
4,2020-03-13T21:19:47,POINT (-97.74620 30.35068)


In [16]:
trackpoints.info(show_counts=True)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 477639 entries, 0 to 477638
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype   
---  ------    --------------   -----   
 0   time      477639 non-null  object  
 1   geometry  477639 non-null  geometry
dtypes: geometry(1), object(1)
memory usage: 7.3+ MB


In [17]:
import pandas as pd
trackpoints['time'] = pd.to_datetime(trackpoints['time'])

In [18]:
trackpoints['time'] = trackpoints['time'].dt.tz_localize('America/Chicago')
trackpoints['date'] = trackpoints['time'].dt.normalize()

In [ ]:
trackpoints

In [20]:
tpoints = trackpoints[['date', 'geometry']].drop_duplicates()

In [24]:
# save the trackpoints to a pickle file to be read in by the Personal Data Datasets notebook 
# for processing the summary health data
tpoints.to_pickle('Apple/apple_health_export/tpoints.pkl')